In [1]:
# import libraries
import pandas as pd
import yfinance as yf
import hvplot.pandas
import numpy as np


import warnings
warnings.filterwarnings('ignore')

In [2]:
# download the historical prices of pltr
pltr_df = yf.download(tickers = 'pltr', period='2Y', interval = '1h')
pltr_df

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,PLTR,PLTR,PLTR,PLTR,PLTR
Datetime,,,,,
2023-06-20 13:30:00+00:00,15.850000,16.889999,15.810000,15.990000,33201677
2023-06-20 14:30:00+00:00,15.770000,15.980000,15.580000,15.845000,16655190
2023-06-20 15:30:00+00:00,15.790000,15.816100,15.650000,15.780000,7249003
2023-06-20 16:30:00+00:00,15.920000,15.980000,15.750000,15.790000,7016700
2023-06-20 17:30:00+00:00,15.815000,15.960000,15.740000,15.920000,4606782
...,...,...,...,...,...
2025-06-17 15:30:00+00:00,139.630005,140.099899,139.262695,139.809998,5904712
2025-06-17 16:30:00+00:00,137.410004,139.800003,137.410004,139.630005,8897805


In [3]:
# Remove multilevels of the column headers
pltr_df = pltr_df.droplevel(level = 1, axis = 1)

# Remove the name 'Price' from the headers
pltr_df.columns.name = None

pltr_df

,Close,High,Low,Open,Volume
Datetime,,,,,
2023-06-20 13:30:00+00:00,15.850000,16.889999,15.810000,15.990000,33201677
2023-06-20 14:30:00+00:00,15.770000,15.980000,15.580000,15.845000,16655190
2023-06-20 15:30:00+00:00,15.790000,15.816100,15.650000,15.780000,7249003
2023-06-20 16:30:00+00:00,15.920000,15.980000,15.750000,15.790000,7016700
2023-06-20 17:30:00+00:00,15.815000,15.960000,15.740000,15.920000,4606782
...,...,...,...,...,...
2025-06-17 15:30:00+00:00,139.630005,140.099899,139.262695,139.809998,5904712
2025-06-17 16:30:00+00:00,137.410004,139.800003,137.410004,139.630005,8897805
2025-06-17 17:30:00+00:00,136.949997,137.880005,136.080002,137.410004,10154922


In [4]:
# Round the values of the dataframe to 2 decimal points
pltr_df = round(pltr_df[['Close', 'High', 'Low', 'Open', 'Volume']], 2)

# Display the data
pltr_df

,Close,High,Low,Open,Volume
Datetime,,,,,
2023-06-20 13:30:00+00:00,15.85,16.89,15.81,15.99,33201677
2023-06-20 14:30:00+00:00,15.77,15.98,15.58,15.85,16655190
2023-06-20 15:30:00+00:00,15.79,15.82,15.65,15.78,7249003
2023-06-20 16:30:00+00:00,15.92,15.98,15.75,15.79,7016700
2023-06-20 17:30:00+00:00,15.81,15.96,15.74,15.92,4606782
...,...,...,...,...,...
2025-06-17 15:30:00+00:00,139.63,140.10,139.26,139.81,5904712
2025-06-17 16:30:00+00:00,137.41,139.80,137.41,139.63,8897805
2025-06-17 17:30:00+00:00,136.95,137.88,136.08,137.41,10154922


In [5]:
# Copy pltr_df to a new dataframe for further analysis
signals_df = pltr_df[['Close']]

# visulise the data
signals_df.hvplot()

:Curve   [Datetime]   (Close)

In [6]:
# Set the long and short windows
short_window = 20
long_window = 50

# Obtain the Exponential Moving Average of the Close prices with short and long windows
signals_df['EMA20_Close'] =  round(signals_df['Close'].ewm(span = short_window).mean(), 2)
signals_df['EMA50_Close'] =  round(signals_df['Close'].ewm(span = long_window).mean(), 2)


# view data
signals_df.head()

,Close,EMA20_Close,EMA50_Close
Datetime,,,
2023-06-20 13:30:00+00:00,15.85,15.85,15.85
2023-06-20 14:30:00+00:00,15.77,15.81,15.81
2023-06-20 15:30:00+00:00,15.79,15.80,15.80
2023-06-20 16:30:00+00:00,15.92,15.84,15.83
2023-06-20 17:30:00+00:00,15.81,15.83,15.83


In [7]:
# Obtain the points of buy and sell using the 20 and 50 day exponential moving averages
buy_points = (signals_df['EMA20_Close'] > signals_df['EMA50_Close']) & (signals_df['EMA20_Close'].shift(1) <= signals_df['EMA50_Close'].shift(1))
sell_points = (signals_df['EMA20_Close'] < signals_df['EMA50_Close']) & (signals_df['EMA20_Close'].shift(1) >= signals_df['EMA50_Close'].shift(1))

# Combine the buy and sell points to obtain the all the crossover points of the EMAs
crossover_points = buy_points | sell_points

# Obtain the first crossover point
first_buy_point = crossover_points.idxmax()

# Mark the crossover points of the EMAs with 1s 
signals_df['Signal'] = np.where((signals_df['EMA20_Close'] > signals_df['EMA50_Close']) & 
                                         (signals_df.index > first_buy_point), 1, 0)

# View data
signals_df.head()

,Close,EMA20_Close,EMA50_Close,Signal
Datetime,,,,
2023-06-20 13:30:00+00:00,15.85,15.85,15.85,0
2023-06-20 14:30:00+00:00,15.77,15.81,15.81,0
2023-06-20 15:30:00+00:00,15.79,15.80,15.80,0
2023-06-20 16:30:00+00:00,15.92,15.84,15.83,0
2023-06-20 17:30:00+00:00,15.81,15.83,15.83,0


In [8]:
# Label the exit and entry points with Buy as 1, Sell as -1 and Hold as 0
signals_df['Entry/Exit'] = signals_df['Signal'].diff()
signals_df.dropna(inplace = True)

signals_df.head()

,Close,EMA20_Close,EMA50_Close,Signal,Entry/Exit
Datetime,,,,,
2023-06-20 14:30:00+00:00,15.77,15.81,15.81,0,0.0
2023-06-20 15:30:00+00:00,15.79,15.80,15.80,0,0.0
2023-06-20 16:30:00+00:00,15.92,15.84,15.83,0,0.0
2023-06-20 17:30:00+00:00,15.81,15.83,15.83,0,0.0
2023-06-20 18:30:00+00:00,15.90,15.84,15.84,0,0.0


In [9]:
# Create a function to obtain the dataframe with the dates around the trades alone
def subset_crossover(df, crossovers):
    crossindex = np.where(crossovers)[0]
    row_ranges = []
    for index in crossindex:
        start = max(index-1, 0)
        end = min(index+1, len(df))
        row_ranges.extend(range(start, end))
    
    # Add the last row of the main dataframe
    row_ranges.append(len(df) - 1)
    
    unique_rows = sorted(set(row_ranges))
    
    return df.iloc[unique_rows]


In [10]:
# Call the function to create the dataframe with only the dates around the trades
crossovers_df = subset_crossover(signals_df, crossover_points)
crossovers_df

,Close,EMA20_Close,EMA50_Close,Signal,Entry/Exit
Datetime,,,,,
2023-06-20 16:30:00+00:00,15.92,15.84,15.83,0,0.0
2023-06-20 17:30:00+00:00,15.81,15.83,15.83,0,0.0
2023-06-20 19:30:00+00:00,15.80,15.84,15.83,1,1.0
2023-06-21 13:30:00+00:00,14.90,15.67,15.70,0,-1.0
2023-06-21 14:30:00+00:00,14.68,15.51,15.57,0,0.0
...,...,...,...,...,...
2025-06-05 19:30:00+00:00,119.91,127.44,127.66,0,-1.0
2025-06-06 13:30:00+00:00,124.64,127.17,127.54,0,0.0
2025-06-09 16:30:00+00:00,131.04,127.68,127.59,1,1.0


In [11]:
# Visualise the data with buy and sell points marked on the chart with the close prices
close_prices = signals_df['Close'].hvplot(color = 'lightgray')

ema20  = signals_df['EMA20_Close'].hvplot(color = 'green')

ema50  = signals_df['EMA50_Close'].hvplot(color = 'yellow')

entry = signals_df[signals_df['Entry/Exit'] == 1]['Close'].hvplot.scatter(color = 'blue',
                                                                 marker = '^',
                                                                 legend = False,
                                                                 size = 200
                                                                )

exit = signals_df[signals_df['Entry/Exit'] == -1]['Close'].hvplot.scatter(color = 'red',
                                                                 marker = 'v',
                                                                 legend = False,
                                                                 size = 200
                                                                )


plot = close_prices * ema20 * ema50 * entry * exit

plot.opts(height = 500,
         width = 1000,
         title = 'Entry Exit plot on Close Prices based on EMA',
         ylabel = 'Price in $')

:Overlay
   .Curve.Close       :Curve   [Datetime]   (Close)
   .Curve.EMA20_Close :Curve   [Datetime]   (EMA20_Close)
   .Curve.EMA50_Close :Curve   [Datetime]   (EMA50_Close)
   .Scatter.Close.I   :Scatter   [Datetime]   (Close)
   .Scatter.Close.II  :Scatter   [Datetime]   (Close)

In [12]:
# Set up values for initial capital and size of position that would be taken in the trades to simulate the trading
initial_capital = 100000
share_size = 150

In [13]:
# Create a column that shows the size of the postion when executing the trade
signals_df['Share_Size'] = abs(signals_df['Entry/Exit'] * share_size)

# View data
crossovers_df = subset_crossover(signals_df, crossover_points)
crossovers_df


,Close,EMA20_Close,EMA50_Close,Signal,Entry/Exit,Share_Size
Datetime,,,,,,
2023-06-20 16:30:00+00:00,15.92,15.84,15.83,0,0.0,0.0
2023-06-20 17:30:00+00:00,15.81,15.83,15.83,0,0.0,0.0
2023-06-20 19:30:00+00:00,15.80,15.84,15.83,1,1.0,150.0
2023-06-21 13:30:00+00:00,14.90,15.67,15.70,0,-1.0,150.0
2023-06-21 14:30:00+00:00,14.68,15.51,15.57,0,0.0,0.0
...,...,...,...,...,...,...
2025-06-05 19:30:00+00:00,119.91,127.44,127.66,0,-1.0,150.0
2025-06-06 13:30:00+00:00,124.64,127.17,127.54,0,0.0,0.0
2025-06-09 16:30:00+00:00,131.04,127.68,127.59,1,1.0,150.0


In [14]:
#### Define the position taken in each trade
signals_df['Position'] = signals_df['Entry/Exit'] * signals_df['Share_Size']

crossovers_df = subset_crossover(signals_df, crossover_points)
crossovers_df.head(20)

,Close,EMA20_Close,EMA50_Close,Signal,Entry/Exit,Share_Size,Position
Datetime,,,,,,,
2023-06-20 16:30:00+00:00,15.92,15.84,15.83,0,0.0,0.0,0.0
2023-06-20 17:30:00+00:00,15.81,15.83,15.83,0,0.0,0.0,0.0
2023-06-20 19:30:00+00:00,15.80,15.84,15.83,1,1.0,150.0,150.0
2023-06-21 13:30:00+00:00,14.90,15.67,15.70,0,-1.0,150.0,-150.0
2023-06-21 14:30:00+00:00,14.68,15.51,15.57,0,0.0,0.0,0.0
2023-06-28 13:30:00+00:00,15.18,14.44,14.43,1,1.0,150.0,150.0
2023-06-28 14:30:00+00:00,15.11,14.50,14.46,1,0.0,0.0,0.0
2023-07-21 19:30:00+00:00,16.43,17.02,17.03,0,-1.0,150.0,-150.0
2023-07-24 13:30:00+00:00,16.50,16.97,17.01,0,0.0,0.0,0.0


In [15]:
# Calculate the portfolio holdings in each trade
signals_df['Portfolio_Holdings'] = signals_df['Position'] * signals_df['Close']

crossovers_df = subset_crossover(signals_df, crossover_points)
crossovers_df.head(20)

,Close,EMA20_Close,EMA50_Close,Signal,Entry/Exit,Share_Size,Position,Portfolio_Holdings
Datetime,,,,,,,,
2023-06-20 16:30:00+00:00,15.92,15.84,15.83,0,0.0,0.0,0.0,0.0
2023-06-20 17:30:00+00:00,15.81,15.83,15.83,0,0.0,0.0,0.0,0.0
2023-06-20 19:30:00+00:00,15.80,15.84,15.83,1,1.0,150.0,150.0,2370.0
2023-06-21 13:30:00+00:00,14.90,15.67,15.70,0,-1.0,150.0,-150.0,-2235.0
2023-06-21 14:30:00+00:00,14.68,15.51,15.57,0,0.0,0.0,0.0,0.0
2023-06-28 13:30:00+00:00,15.18,14.44,14.43,1,1.0,150.0,150.0,2277.0
2023-06-28 14:30:00+00:00,15.11,14.50,14.46,1,0.0,0.0,0.0,0.0
2023-07-21 19:30:00+00:00,16.43,17.02,17.03,0,-1.0,150.0,-150.0,-2464.5
2023-07-24 13:30:00+00:00,16.50,16.97,17.01,0,0.0,0.0,0.0,0.0


In [16]:
# Obtain the cash reserve of the pportfolio
signals_df['Portfolio_Cash'] = initial_capital - (signals_df['Close'] * signals_df['Position']).cumsum()

crossovers_df = subset_crossover(signals_df, crossover_points)
crossovers_df

,Close,EMA20_Close,EMA50_Close,Signal,Entry/Exit,Share_Size,Position,Portfolio_Holdings,Portfolio_Cash
Datetime,,,,,,,,,
2023-06-20 16:30:00+00:00,15.92,15.84,15.83,0,0.0,0.0,0.0,0.0,100000.0
2023-06-20 17:30:00+00:00,15.81,15.83,15.83,0,0.0,0.0,0.0,0.0,100000.0
2023-06-20 19:30:00+00:00,15.80,15.84,15.83,1,1.0,150.0,150.0,2370.0,97630.0
2023-06-21 13:30:00+00:00,14.90,15.67,15.70,0,-1.0,150.0,-150.0,-2235.0,99865.0
2023-06-21 14:30:00+00:00,14.68,15.51,15.57,0,0.0,0.0,0.0,0.0,99865.0
...,...,...,...,...,...,...,...,...,...
2025-06-05 19:30:00+00:00,119.91,127.44,127.66,0,-1.0,150.0,-150.0,-17986.5,111133.0
2025-06-06 13:30:00+00:00,124.64,127.17,127.54,0,0.0,0.0,0.0,0.0,111133.0
2025-06-09 16:30:00+00:00,131.04,127.68,127.59,1,1.0,150.0,150.0,19656.0,91477.0


In [17]:
# Calculate the total value of the portfolio at each time
# signals_df['Portfolio_Total'] = initial_capital
# signals_df['Portfolio_Total'] = np.where(signals_df['Signal'] == 1, (signals_df['Portfolio_Cash'] + (signals_df['Close'] * share_size)), signals_df['Portfolio_Total'].shift(1))

value_list = [initial_capital]

for i in range(1, len(signals_df)):
    if signals_df.iloc[i]['Signal'] == 1 and signals_df.iloc[i]['Portfolio_Holdings'] != 0:
        value =  signals_df.iloc[i]['Portfolio_Holdings'] + signals_df.iloc[i]['Portfolio_Cash']
    
    elif signals_df.iloc[i]['Signal'] == 1 and signals_df.iloc[i]['Portfolio_Holdings'] == 0:
        value = signals_df.iloc[i]['Portfolio_Cash'] + (share_size * signals_df.iloc[i]['Close'])
        
    else:
        value = value_list[-1]
        
    value_list.append(value)
    

signals_df['Portfolio_Total'] = value_list
signals_df.head(50)


crossovers_df = subset_crossover(signals_df, crossover_points)
crossovers_df

,Close,EMA20_Close,EMA50_Close,Signal,Entry/Exit,Share_Size,Position,Portfolio_Holdings,Portfolio_Cash,Portfolio_Total
Datetime,,,,,,,,,,
2023-06-20 16:30:00+00:00,15.92,15.84,15.83,0,0.0,0.0,0.0,0.0,100000.0,100000.0
2023-06-20 17:30:00+00:00,15.81,15.83,15.83,0,0.0,0.0,0.0,0.0,100000.0,100000.0
2023-06-20 19:30:00+00:00,15.80,15.84,15.83,1,1.0,150.0,150.0,2370.0,97630.0,100000.0
2023-06-21 13:30:00+00:00,14.90,15.67,15.70,0,-1.0,150.0,-150.0,-2235.0,99865.0,100000.0
2023-06-21 14:30:00+00:00,14.68,15.51,15.57,0,0.0,0.0,0.0,0.0,99865.0,100000.0
...,...,...,...,...,...,...,...,...,...,...
2025-06-05 19:30:00+00:00,119.91,127.44,127.66,0,-1.0,150.0,-150.0,-17986.5,111133.0,111203.5
2025-06-06 13:30:00+00:00,124.64,127.17,127.54,0,0.0,0.0,0.0,0.0,111133.0,111203.5
2025-06-09 16:30:00+00:00,131.04,127.68,127.59,1,1.0,150.0,150.0,19656.0,91477.0,111133.0


In [18]:
signals_df['Signal'].value_counts()

Signal
1    2118
0    1368
Name: count, dtype: int64

In [19]:
signals_df.head(10)

,Close,EMA20_Close,EMA50_Close,Signal,Entry/Exit,Share_Size,Position,Portfolio_Holdings,Portfolio_Cash,Portfolio_Total
Datetime,,,,,,,,,,
2023-06-20 14:30:00+00:00,15.77,15.81,15.81,0,0.0,0.0,0.0,0.0,100000.0,100000.0
2023-06-20 15:30:00+00:00,15.79,15.80,15.80,0,0.0,0.0,0.0,0.0,100000.0,100000.0
2023-06-20 16:30:00+00:00,15.92,15.84,15.83,0,0.0,0.0,0.0,0.0,100000.0,100000.0
2023-06-20 17:30:00+00:00,15.81,15.83,15.83,0,0.0,0.0,0.0,0.0,100000.0,100000.0
2023-06-20 18:30:00+00:00,15.90,15.84,15.84,0,0.0,0.0,0.0,0.0,100000.0,100000.0
2023-06-20 19:30:00+00:00,15.80,15.84,15.83,1,1.0,150.0,150.0,2370.0,97630.0,100000.0
2023-06-21 13:30:00+00:00,14.90,15.67,15.70,0,-1.0,150.0,-150.0,-2235.0,99865.0,100000.0
2023-06-21 14:30:00+00:00,14.68,15.51,15.57,0,0.0,0.0,0.0,0.0,99865.0,100000.0
2023-06-21 15:30:00+00:00,14.44,15.35,15.43,0,0.0,0.0,0.0,0.0,99865.0,100000.0


In [20]:
# Obtain the Daily returns of the portfolio and clean the data
signals_df['Portfolio_Daily_Returns'] = signals_df['Portfolio_Total'].pct_change()
signals_df.dropna(inplace = True)

crossovers_df = subset_crossover(signals_df, crossover_points)
crossovers_df

,Close,EMA20_Close,EMA50_Close,Signal,Entry/Exit,Share_Size,Position,Portfolio_Holdings,Portfolio_Cash,Portfolio_Total,Portfolio_Daily_Returns
Datetime,,,,,,,,,,,
2023-06-20 17:30:00+00:00,15.81,15.83,15.83,0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.000000
2023-06-20 18:30:00+00:00,15.90,15.84,15.84,0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.000000
2023-06-21 13:30:00+00:00,14.90,15.67,15.70,0,-1.0,150.0,-150.0,-2235.0,99865.0,100000.0,0.000000
2023-06-21 14:30:00+00:00,14.68,15.51,15.57,0,0.0,0.0,0.0,0.0,99865.0,100000.0,0.000000
2023-06-21 15:30:00+00:00,14.44,15.35,15.43,0,0.0,0.0,0.0,0.0,99865.0,100000.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
2025-06-06 13:30:00+00:00,124.64,127.17,127.54,0,0.0,0.0,0.0,0.0,111133.0,111203.5,0.000000
2025-06-06 14:30:00+00:00,124.76,126.94,127.43,0,0.0,0.0,0.0,0.0,111133.0,111203.5,0.000000
2025-06-09 17:30:00+00:00,131.54,128.05,127.74,1,0.0,0.0,0.0,0.0,91477.0,111208.0,0.000675


In [21]:
signals_df['Cumulative_Daily_Returns'] = (1 + signals_df['Portfolio_Daily_Returns']).cumprod() - 1

crossovers_df = subset_crossover(signals_df, crossover_points)
crossovers_df

,Close,EMA20_Close,EMA50_Close,Signal,Entry/Exit,Share_Size,Position,Portfolio_Holdings,Portfolio_Cash,Portfolio_Total,Portfolio_Daily_Returns,Cumulative_Daily_Returns
Datetime,,,,,,,,,,,,
2023-06-20 17:30:00+00:00,15.81,15.83,15.83,0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.000000,0.000000
2023-06-20 18:30:00+00:00,15.90,15.84,15.84,0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.000000,0.000000
2023-06-21 13:30:00+00:00,14.90,15.67,15.70,0,-1.0,150.0,-150.0,-2235.0,99865.0,100000.0,0.000000,0.000000
2023-06-21 14:30:00+00:00,14.68,15.51,15.57,0,0.0,0.0,0.0,0.0,99865.0,100000.0,0.000000,0.000000
2023-06-21 15:30:00+00:00,14.44,15.35,15.43,0,0.0,0.0,0.0,0.0,99865.0,100000.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-06 13:30:00+00:00,124.64,127.17,127.54,0,0.0,0.0,0.0,0.0,111133.0,111203.5,0.000000,0.112035
2025-06-06 14:30:00+00:00,124.76,126.94,127.43,0,0.0,0.0,0.0,0.0,111133.0,111203.5,0.000000,0.112035
2025-06-09 17:30:00+00:00,131.54,128.05,127.74,1,0.0,0.0,0.0,0.0,91477.0,111208.0,0.000675,0.112080


In [22]:
signals_df['Cumulative_Daily_Returns'].hvplot()

:Curve   [Datetime]   (Cumulative_Daily_Returns)

In [23]:
# Plot the buys and sells on the graph
exit = signals_df[signals_df['Entry/Exit'] == -1]['Portfolio_Total'].hvplot.scatter(color = 'red',
                                       marker = 'v',
                                       legend = False,
                                       width = 1000,
                                       height = 500,
                                       size = 200, ylabel = 'Price in $')
exit

entry = signals_df[signals_df['Entry/Exit'] == 1]['Portfolio_Total'].hvplot.scatter(color = 'green',
                                       marker = '^',
                                       legend = False,
                                       width = 1000,
                                       height = 500,
                                       size = 200, ylabel = 'Price in $')

entry



portfolio_price_chart = signals_df['Portfolio_Total'].hvplot(color = 'lightgray')

entry_exit_chart = portfolio_price_chart * entry * exit 

entry_exit_chart.opts(title = 'Entry Exit Plot on the Portfolio Cumulative Returns',
                     height = 500,
                     width = 1000)

:Overlay
   .Curve.Portfolio_Total      :Curve   [Datetime]   (Portfolio_Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Datetime]   (Portfolio_Total)
   .Scatter.Portfolio_Total.II :Scatter   [Datetime]   (Portfolio_Total)

## Calculating Metrics

In [24]:
metics = ['Annualized Returns',
          'Cumulative Returns',
          'Annualized Volatility',
          'Sharpe Ratio',
          'Sortino Ratio']

evaluation_df = pd.DataFrame(columns = ['Backtest'], index = metics)
evaluation_df

,Backtest
Annualized Returns,NaN
Cumulative Returns,NaN
Annualized Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [25]:
# Add the first four respective data to the data frame
evaluation_df.loc['Annualized Returns'] = signals_df['Portfolio_Daily_Returns'].mean() * 252
evaluation_df.loc['Cumulative Returns'] = signals_df['Cumulative_Daily_Returns'][-1]
evaluation_df.loc['Annualized Volatility'] = signals_df['Portfolio_Daily_Returns'].std() * np.sqrt(252)
evaluation_df.loc['Sharpe Ratio'] = (signals_df['Portfolio_Daily_Returns'].mean() * 252) / (signals_df['Portfolio_Daily_Returns'].std() * np.sqrt(252))

# Display the data
evaluation_df

,Backtest
Annualized Returns,0.008552
Cumulative Returns,0.12285
Annualized Volatility,0.01863
Sharpe Ratio,0.459045
Sortino Ratio,NaN


In [26]:
# Convert the returns to numneric and drop 'NaN's for calulation
daily_returns = pd.to_numeric(signals_df['Portfolio_Daily_Returns']).dropna()


# Calculate average daily return
average_return = daily_returns.mean()

# Calculate downside returns (only negative returns)
downside_returns = daily_returns[daily_returns < 0]

# Calculate the downside standard deviation
downside_std = downside_returns.std()

# Calculate the Sortino Ratio
sortino_ratio = (average_return / downside_std) * np.sqrt(252)
sortino_ratio

0.3943130510541524

In [27]:
# Obtain the sortino ratio and add it to the evaluation dataframe
evaluation_df.loc['Sortino Ratio'] = sortino_ratio
evaluation_df

,Backtest
Annualized Returns,0.008552
Cumulative Returns,0.12285
Annualized Volatility,0.01863
Sharpe Ratio,0.459045
Sortino Ratio,0.394313


## Performance analysis of the strategy

In [28]:
# Create the dataframe specifying the features of the trades along with its profit and loss
performance_data = []

    
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_share_price = row['Close']
        share_size = abs(row['Position'])
        entry_portfolio_holdings = row['Portfolio_Holdings']

    elif row['Entry/Exit'] == -1 and entry_date is not None:
        exit_date = index
        exit_share_price = row['Close']
        share_size = abs(row['Position'])
        exit_portfolio_holdings = abs(row['Portfolio_Holdings'])
        profit_loss = exit_portfolio_holdings - entry_portfolio_holdings

        performance_data.append({
            'Stock': 'PLTR',
            'Entry Date': entry_date,
            'Exit Date': exit_date,
            'Entry Price': entry_share_price,
            'Exit Price': exit_share_price,
            'Shares': share_size,
            'Entry Portfolio Holding': entry_portfolio_holdings,
            'Exit Portfolio Holding': exit_portfolio_holdings,
            'Profit/Loss': profit_loss
        })

        

performance_data_df = pd.DataFrame(performance_data)

performance_data_df

,Stock,Entry Date,Exit Date,Entry Price,Exit Price,Shares,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,PLTR,2023-06-20 19:30:00+00:00,2023-06-21 13:30:00+00:00,15.80,14.90,150.0,2370.0,2235.0,-135.0
1,PLTR,2023-06-28 13:30:00+00:00,2023-07-21 19:30:00+00:00,15.18,16.43,150.0,2277.0,2464.5,187.5
2,PLTR,2023-07-28 17:30:00+00:00,2023-08-07 16:30:00+00:00,17.69,17.66,150.0,2653.5,2649.0,-4.5
3,PLTR,2023-08-29 16:30:00+00:00,2023-09-07 14:30:00+00:00,15.43,14.84,150.0,2314.5,2226.0,-88.5
4,PLTR,2023-09-08 14:30:00+00:00,2023-09-18 13:30:00+00:00,15.33,15.36,150.0,2299.5,2304.0,4.5
5,PLTR,2023-09-28 14:30:00+00:00,2023-10-20 13:30:00+00:00,15.55,16.29,150.0,2332.5,2443.5,111.0
6,PLTR,2023-11-02 15:30:00+00:00,2023-11-24 16:30:00+00:00,17.58,19.27,150.0,2637.0,2890.5,253.5
7,PLTR,2023-11-30 14:30:00+00:00,2023-12-04 15:30:00+00:00,20.08,18.56,150.0,3012.0,2784.0,-228.0
8,PLTR,2023-12-15 15:30:00+00:00,2023-12-18 20:30:00+00:00,18.65,17.84,150.0,2797.5,2676.0,-121.5
9,PLTR,2024-01-22 14:30:00+00:00,2024-01-25 19:30:00+00:00,18.09,16.41,150.0,2713.5,2461.5,-252.0
